In [1]:
def screen_capture():
  import pyscreenshot as ImageGrab
  import time
    
  images_folder = "captured_images/9/"
    
  for i in range(20):
    time.sleep(5)
    im=ImageGrab.grab(bbox=(60,170,400,550))
    print("saved...",i)
    im.save(images_folder+str(i)+'.png')
    print("clear screen and redraw again....")
screen_capture()


In [5]:
import numpy as np
def create_label(image_name):
    if image_name == '0':
        return np.array([1,0,0,0,0,0,0,0,0,0])
    elif image_name == '1':
        return np.array([0,1,0,0,0,0,0,0,0,0])
    elif image_name == '2':
        return np.array([0,0,1,0,0,0,0,0,0,0])
    elif image_name == '3':
        return np.array([0,0,0,1,0,0,0,0,0,0])
    elif image_name == '4':
        return np.array([0,0,0,0,1,0,0,0,0,0])
    elif image_name == '5':
        return np.array([0,0,0,0,0,1,0,0,0,0])
    elif image_name == '6':
        return np.array([0,0,0,0,0,0,1,0,0,0])
    elif image_name == '7':
        return np.array([0,0,0,0,0,0,0,1,0,0])
    elif image_name == '8':
        return np.array([0,0,0,0,0,0,0,0,1,0])
    elif image_name == '9':
        return np.array([0,0,0,0,0,0,0,0,0,1])

In [6]:
import os
import cv2
from random import shuffle
from tqdm import tqdm
def create_data():
    data=[]
    for folder in tqdm(os.listdir("captured_images")):
        for img in os.listdir("captured_images/"+folder):
            path=os.path.join("captured_images",folder,img)
            img_data = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            try:
                img_data = cv2.resize(img_data,(28,28))
            except cv2.error as e:
                continue
            data.append([np.array(img_data),create_label(folder)])
    shuffle(data)
    return data

In [7]:
data=create_data()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.80it/s]


In [8]:
#training and testing part
train=data[:800]
test=data[800:]
x_train=np.array([i[0] for i in train]).reshape(-1,28,28,1)
y_train=[i[1] for i in train]
x_test=np.array([i[0] for i in test]).reshape(-1,28,28,1)
y_test=[i[1] for i in test]

In [9]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow.compat.v1 as tf
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [ ]:
tf.reset_default_graph()
convnet = input_data(shape=[28,28, 1], name='input') 
convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8) #prevent a model from overfitting
convnet = fully_connected(convnet, 10, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=0.001, 
                     loss='categorical_crossentropy', name='targets') 
model = tflearn.DNN(convnet, tensorboard_verbose=1) 
model.fit({'input': x_train}, {'targets': y_train}, n_epoch=12,
          validation_set=({'input': x_test}, {'targets': y_test}),
          show_metric=True)

Training Step: 129  | total loss: 0.23783 | time: 1.065s
| Adam | epoch: 010 | loss: 0.23783 - acc: 0.9280 -- iter: 768/800
Training Step: 130  | total loss: 0.26059 | time: 2.176s
| Adam | epoch: 010 | loss: 0.26059 - acc: 0.9196 | val_loss: 0.45024 - val_acc: 0.8713 -- iter: 800/800
--


In [ ]:
def create_test_data():
    data=[]
    for img in tqdm(os.listdir("new_images")):
        path=os.path.join("new_images",img)
        img_num=img.split('.')[0]
        img_data = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        try:
            img_data = cv2.resize(img_data,(28,28))
        except cv2.error as e:
            continue
        data.append([np.array(img_data),img_num])
    shuffle(data)
    return data
    

In [ ]:
test_data = create_test_data()

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10,10))
for num, data in enumerate(test_data[:10]):
    img_data = data[0]
    y = fig.add_subplot(5,5, num + 1)
    orig = img_data
    data = img_data.reshape(28,28, 1)
    model_out = model.predict([data])
    str_label = "Prediction: " + str(np.argmax(model_out))

    y.imshow(orig, cmap='gray')
    plt.title(str_label)
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()
